In [1]:
import requests
import json
from tqdm import tqdm
from pathlib import Path
import sys
sys.path.append('..')
import paths


In [2]:
""" simulate curl --header "Content-Type: application/json" \
  --request POST \
  --data '{"text":"Who painted The Storm on the Sea of Galilee?"}' \
  'https://labs.tib.eu/falcon/falcon2/api?mode=long&k=50'
"""
#use a function receiving the text
def get_falcon(text):
    #url of the api
    url = 'https://labs.tib.eu/falcon/falcon2/api?mode=long&k=50'
    #data to be sent
    data = {'text': text}
    #headers
    headers = {'Content-Type': 'application/json'}
    #send the request
    response = requests.post(url, headers=headers, data=json.dumps(data))
    #return the response
    return response.json()

In [3]:
res=get_falcon("Who painted The Storm on the Sea of Galilee?")
res

{'entities_wikidata': [{'URI': 'http://www.wikidata.org/entity/Q2246489',
   'surface form': 'Storm on the Sea of Galilee'},
  {'URI': 'http://www.wikidata.org/entity/Q2246489',
   'surface form': 'Storm on the Sea of Galilee'},
  {'URI': 'http://www.wikidata.org/entity/Q21712166',
   'surface form': 'Storm on the Sea of Galilee'},
  {'URI': 'http://www.wikidata.org/entity/Q27898358',
   'surface form': 'Storm on the Sea of Galilee'},
  {'URI': 'http://www.wikidata.org/entity/Q20355284',
   'surface form': 'Storm on the Sea of Galilee'},
  {'URI': 'http://www.wikidata.org/entity/Q18748474',
   'surface form': 'Storm on the Sea of Galilee'},
  {'URI': 'http://www.wikidata.org/entity/Q20808844',
   'surface form': 'Storm on the Sea of Galilee'},
  {'URI': 'http://www.wikidata.org/entity/Q28778027',
   'surface form': 'Storm on the Sea of Galilee'},
  {'URI': 'http://www.wikidata.org/entity/Q17321007',
   'surface form': 'Storm on the Sea of Galilee'},
  {'URI': 'http://www.wikidata.org/e

In [4]:
annotation_path=paths.COMBINED_ANNOTATIONS_PATH
annotation_path.exists()

True

In [5]:
annotation_path

PosixPath('/hpi/fs00/home/alejandro.sierra/MELArt/output_files/melart_annotations.json')

In [6]:
annotations=json.loads(open(annotation_path).read())
#test set when split="test"
test_set={x:y for x,y in annotations.items() if y['split']=='test'}
len(test_set)

100

In [7]:
test_set[list(test_set.keys())[0]]

{'P180': ['http://www.wikidata.org/entity/Q345',
  'http://www.wikidata.org/entity/Q942467'],
 'P18': ['http://commons.wikimedia.org/wiki/Special:FilePath/Cimabue%20032.jpg'],
 'visual_sentences': ['It is also stylistically earlier to that work, being painted without pseudo-perspective, and having the angels around the Virgin simply placed one above the other, rather than being spatially arranged.',
  'The throne is similar to the Maestà painted by Cimabue in the Basilica of San Francesco di Assisi (1288–1292).'],
 'contextual_sentences': ['The Maestà is a painting by the Italian artist   Cimabue, painted around 1280 and housed in Musée du Louvre of Paris, France.',
  'The work is considered to be from around 1280, thus preceding the Santa Trinita Maestà.',
  'This work established a new canon for the Madonna with Child theme, which was subsequently used by other painters, such as Duccio di Buoninsegna in his Rucellai Maestà.'],
 'title': 'Maestà (Cimabue)',
 'img_url': 'http://commons

In [8]:
path_falcon_res=Path("falcon_res.json")

In [9]:
falcon_res={}
if path_falcon_res.exists():
    falcon_res=json.loads(open(path_falcon_res).read())

In [10]:
for qid,anns in tqdm(test_set.items()):
    if not qid in falcon_res:
        falcon_res[qid]=[]
    sentences=[]
    for sentence_field,matches_field in [('visual_sentences','visual_el_matches'),('contextual_sentences','contextual_el_matches')]:
        for i,sentence in enumerate(anns[sentence_field]):
            if anns[matches_field][i]:
                sentences.append(sentence)
    for sentence in sentences:
        text=sentence
        if any([text==x['text'] and x['falcon']!="Error" for x in falcon_res[qid]]):
            continue
        try:
            res=get_falcon(text)
        except:
            res="Error"
            print('Error with:',text)
        for x in falcon_res[qid]:
            if x['text']==text:
                falcon_res[qid].remove(x)
        falcon_res[qid].append({'text':text,'falcon':res})

with open('falcon_res.json','w') as f:
    json.dump(falcon_res,f)

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [01:03<00:00,  1.57it/s]
